In [ ]:
import torch
import numpy as np
from sklearn.metrics import average_precision_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
import time

# Function to calculate MAP
def calculate_map(targets, predictions):
    print("Calculating Mean Average Precision (MAP)...")
    map_score = average_precision_score(targets, predictions, average="macro")
    print(f"MAP calculation complete: {map_score:.4f}")
    return map_score

# Load data
print("Loading data...")
start_time = time.time()
train_data = torch.load('/scratch/rd3629/ml-project/AudioSet-classification/Data/train_representations_firstlayer.pt')
test_data = torch.load('/scratch/rd3629/ml-project/AudioSet-classification/Data/test_representations_firstlayer.pt')
train_labels = torch.load('/scratch/rd3629/ml-project/AudioSet-classification/Data/train/labels.pt')
test_labels = torch.load('/scratch/rd3629/ml-project/AudioSet-classification/Data/test/labels.pt')
print(f"Data loaded in {time.time() - start_time:.2f} seconds.")

# Check shapes
print("Inspecting data shapes...")
print(f"Train data shape: {train_data.shape}")  # [20550, 496, 768]
print(f"Test data shape: {test_data.shape}")    # [18886, 496, 768]
print(f"Train labels shape: {train_labels.shape}")  # [20550, 527]
print(f"Test labels shape: {test_labels.shape}")    # [18886, 527]

# Apply max pooling along the time dimension (496)
print("Applying max pooling along the time dimension (496)...")
start_time = time.time()

train_data = torch.max(train_data, dim=1).values.numpy()  # Max pooling across dimension 1 (time)
test_data = torch.max(test_data, dim=1).values.numpy()    # Max pooling across dimension 1 (time)
train_labels = train_labels.numpy()  # [20550, 527]
test_labels = test_labels.numpy()    # [18886, 527]
print(f"Max pooled train data shape: {train_data.shape}")  # [20550, 768]
print(f"Max pooled test data shape: {test_data.shape}")    # [18886, 768]
print(f"Max pooling completed in {time.time() - start_time:.2f} seconds.")

# Train a Logistic Regression Model with OneVsRestClassifier
print("Initializing Logistic Regression model with OneVsRestClassifier...")
start_time = time.time()
model = OneVsRestClassifier(
    LogisticRegression(
        max_iter=1000,  # Set a very low maximum number of iterations to avoid convergence
        solver='saga',
        verbose=1,
        tol=1e-2  # Set a higher tolerance to relax the stopping criterion
    )
    , n_jobs=-1
)
print("Training Logistic Regression model (no convergence required)...")
model.fit(train_data, train_labels)
print(f"Model training complete in {time.time() - start_time:.2f} seconds.")

# Predict probabilities on the test set
print("Generating predictions on test data...")
start_time = time.time()
test_predictions = model.predict_proba(test_data)  # [18886, 527]
print(f"Predictions generated in {time.time() - start_time:.2f} seconds.")

# Metrics
print("Calculating evaluation metrics...")
start_time = time.time()
test_map = calculate_map(test_labels, test_predictions)
test_f1 = f1_score(test_labels, (test_predictions > 0.5).astype(float), average="micro")
print(f"F1-Score calculation complete: {test_f1:.4f}")
print(f"Metrics calculated in {time.time() - start_time:.2f} seconds.")

# Print results
print("\n--- Evaluation Results ---")
print(f"Test MAP: {test_map:.4f}")
print(f"Test F1-Score: {test_f1:.4f}")


Loading data...


/state/partition1/job-54630313/ipykernel_1716894/2272478307.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_data = torch.load('/scratch/rd3629/ml-project/AudioSet

Data loaded in 34.72 seconds.
Inspecting data shapes...
Train data shape: torch.Size([20550, 496, 768])
Test data shape: torch.Size([18886, 496, 768])
Train labels shape: torch.Size([20550, 527])
Test labels shape: torch.Size([18886, 527])
Applying max pooling along the time dimension (496)...
Max pooled train data shape: (20550, 768)
Max pooled test data shape: (18886, 768)
Max pooling completed in 17.71 seconds.
Initializing Logistic Regression model with OneVsRestClassifier...
Training Logistic Regression model (no convergence required)...
Epoch 1, change: 1.00000000
Epoch 1, change: 1.00000000
Epoch 2, change: 0.28372264
Epoch 2, change: 0.35820341
Epoch 3, change: 0.13491730
Epoch 3, change: 0.17236398
Epoch 4, change: 0.09306689
Epoch 4, change: 0.15872677
Epoch 5, change: 0.05792713
Epoch 5, change: 0.11637118
Epoch 6, change: 0.04973323
Epoch 6, change: 0.09317320
Epoch 7, change: 0.03957486
Epoch 7, change: 0.07894008
Epoch 8, change: 0.03271864
Epoch 8, change: 0.06504557
Epo

In [ ]:
# Find a correct test point
test_predictions = model.predict_proba(test_data)  # Get probabilities
binary_predictions = (test_predictions > 0.5).astype(float)  # Threshold predictions

print("Searching for a correct test point where predictions match true labels...")
correct_index = None
for i in range(len(test_labels)):
    if np.array_equal(binary_predictions[i], test_labels[i]):  # Check if predictions match true labels
        correct_index = i
        break

if correct_index is not None:
    print(f"Correct test point found at index {correct_index}")
    predicted_labels = binary_predictions[correct_index]
    true_labels = test_labels[correct_index]
    print("\n--- Correct Test Point ---")
    print(f"Index: {correct_index}")
    print(f"Predicted Labels: {predicted_labels}")
    print(f"True Labels: {true_labels}")
else:
    print("No completely correct test point found.")
